In [12]:
### --- BasicReduction --- ###
### --- Owner: Barbara Joachimczyk --- ###
### ------------------------------------------------------------------------------------------------------ ###


from astropy.io import fits
from astropy.coordinates import EarthLocation, AltAz, SkyCoord
from astropy.time import Time
from astropy import units as u
from astropy.wcs import WCS
from astropy.visualization import ZScaleInterval
from astroquery.astrometry_net import AstrometryNet
from astroquery.astrometry_net import conf
import matplotlib.pyplot as plt 
import numpy as np 
from os import listdir, makedirs
from os.path import isfile, join, exists
from os import renames
import glob
import time
from itertools import chain







# ---  Zmienne globalne --- 

Observatory = EarthLocation(lat=53.093999*u.deg, lon=18.555925*u.deg, height=70*u.m)














        

#                     FUNKCJE                   










# w tym miejscu mam posortowane pliki i zbudowane masterramki, które przechowuje obiekt masterFrames()






### REDUKCJA DANYCH Z RAMEK NAUKOWYCH 













masterFrames = Masters()  

SortBDFFiles(r"D:\!STUDIA\Piwnice\!DANE\20220106\Przemek")
CreateMasterFrames(r"D:\!STUDIA\Piwnice\!DANE\20220106\Przemek")
CalculateScienceFrames(r"D:\!STUDIA\Piwnice\!DANE\20220106\Przemek")


        





Masterbias created.
Masterdark created.
[ERROR] Dark not found.


AttributeError: 'NoneType' object has no attribute 'data'